In [1]:
NB_TRAINING_SAMPLES=-1 # -1 means "all"
NB_TEST_SAMPLES=-1
NB_EPOCHS=1
VERBOSITY=0

# LOADING DATA

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models, Model
import numpy as np
import warnings
warnings.filterwarnings('ignore')

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

if NB_TRAINING_SAMPLES!=-1:
    train_images=train_images[:NB_TRAINING_SAMPLES]
    train_labels=train_labels[:NB_TRAINING_SAMPLES]
if NB_TEST_SAMPLES!=-1:
    test_images=test_images[:NB_TEST_SAMPLES]
    test_labels=test_labels[:NB_TEST_SAMPLES]

train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels=to_categorical(train_labels).astype(np.float32)
test_labels=to_categorical(test_labels).astype(np.float32)

print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(test_labels.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


# Util functions

TODO: https://github.com/hollance/reliability-diagrams/blob/master/reliability_diagrams.py 

In [3]:
import tensorflow_probability as tfp
import sklearn
import tensorflow as tf
from tensorflow.keras import Input, Model, optimizers, layers
import numpy as np

In [7]:
def CE(target,output,epsilon=1e-7):
    bce = target * np.log( output +epsilon) + (1-target) * np.log(  1-output+epsilon)
    return -np.mean(bce)
def softmax(x):
  return np.exp(x) /  np.sum(np.exp(x), axis=-1, keepdims=True)

  
def brier_score(y,y_pred):
    b=(y_pred-y)**2
    return np.mean( np.sum( b , axis=-1 ) )
def evaluate(y,y_pred,out_type="html"): #out_type in set(["html","array","both"])
  
  labels_predicted=np.argmax(y_pred,axis=1).astype(np.int32)
  labels_gt=np.argmax(y,axis=1).astype(np.int32)

  acc=np.mean(labels_predicted==labels_gt)
  nll=CE(y,y_pred)
  
  try:
    num_bins=15 #usual value in papers
    ece=tfp.stats.expected_calibration_error(
      num_bins, logits=y_pred, labels_true=labels_gt, name=None
    ).numpy()
  except:
    ece=np.nan

  try:
    brier=brier_score(y,y_pred)
  except:
    brier=np.nan
  res_array=np.round([acc,nll, ece, brier],4)
  res_html=f"<td>{res_array[0]}</td><td>{res_array[1]}</td><td>{res_array[2]}</td><td>{res_array[3]}</td>"
  
  if out_type=="html":
    return res_html 
  elif out_type=="array":
    return res_array
  elif out_type=="both":
    return res_html, res_array
  else:
    raise ValueError(f"unknown type:{out_type}")


In [5]:
def keras_model_builder(train_images, train_labels, drop_rate=0):
    # Same than https://www.tensorflow.org/tutorials/images/cnn
    inpx=layers.Input(shape=(32, 32, 3))
    x=layers.Conv2D(32, (3, 3), activation='relu')(inpx)
    if drop_rate>0:
      x=layers.Dropout(drop_rate)(x,training=True)
    x=layers.MaxPooling2D((2, 2))(x)
    x=layers.Conv2D(64, (3, 3), activation='relu')(x)
    if drop_rate>0:
      x=layers.Dropout(drop_rate)(x,training=True)
    x=layers.MaxPooling2D((2, 2))(x)
    x=layers.Conv2D(64, (3, 3), activation='relu')(x)
    x=layers.Flatten()(x)
    if drop_rate>0:
      x=layers.Dropout(drop_rate)(x,training=True)
    x=layers.Dense(64, activation='relu')(x)
    if drop_rate>0:
      x=layers.Dropout(drop_rate)(x,training=True)
    x=layers.Dense(10, activation='softmax')(x)
    model=Model(inpx,x)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy())
    history = model.fit(train_images, train_labels, epochs=NB_EPOCHS, 
                    validation_data=(test_images, test_labels),verbose=VERBOSITY)
    return model

# Simple DNN

In [ ]:
model=keras_model_builder(train_images, train_labels)
test_y_pred=model(test_images)
score_info=evaluate(test_labels,test_y_pred)
print(score_info)
del model

[0.5368 0.2005 0.3844 0.5932]


# MC DROPOUT<br>
URL: https://arxiv.org/pdf/1506.02142.pdf

In [13]:
for d in [0.2]: # Value used in the original paper 0.1 0.05 0.005
  model=keras_model_builder(train_images, train_labels, drop_rate=d)
  iters=64 # original paper (10)
  test_y_pred=np.zeros(test_labels.shape)
  for i in range(iters):
      test_y_pred+=model(test_images)
      if (i+1) in set([4, 8, 16, 32]):
        score_info=evaluate(test_labels,test_y_pred/(i+1))
        print(f"<tr><td>MC-drop rate={d}</td><td>1</td><td>{i+1}</td>{score_info}</tr>")
  del model

KeyboardInterrupt: ignored

# ENSEMBLE OF DNN<br>
URL: https://proceedings.neurips.cc/paper/2017/file/9ef2ed4b7fd2c810847ffa5fa85bce38-Paper.pdf

In [9]:
import numpy as np
max_ensemble_size=16
global_preds=np.zeros(test_labels.shape)
for i in range(max_ensemble_size):
  model=keras_model_builder(train_images, train_labels) # create independant training
  global_preds+=model(test_images)
  del model
  ensemble_i_pred=global_preds/(i+1)
  score_info=evaluate(test_labels, ensemble_i_pred)
  print(f"<tr><td>Ensemble</td><td>{i+1}</td><td>{i+1}</td>{score_info}</tr>")

<tr><td>Ensemble</td><td>1</td><td>1</td><td>0.5353</td><td>0.2021</td><td>0.3782</td><td>0.5967</td></tr>
<tr><td>Ensemble</td><td>2</td><td>2</td><td>0.5757</td><td>0.1876</td><td>0.4212</td><td>0.5535</td></tr>
<tr><td>Ensemble</td><td>3</td><td>3</td><td>0.5761</td><td>0.1881</td><td>0.424</td><td>0.5555</td></tr>
<tr><td>Ensemble</td><td>4</td><td>4</td><td>0.5776</td><td>0.1885</td><td>0.4262</td><td>0.5569</td></tr>
<tr><td>Ensemble</td><td>5</td><td>5</td><td>0.5786</td><td>0.1883</td><td>0.4279</td><td>0.5561</td></tr>
<tr><td>Ensemble</td><td>6</td><td>6</td><td>0.5789</td><td>0.1877</td><td>0.4281</td><td>0.5548</td></tr>
<tr><td>Ensemble</td><td>7</td><td>7</td><td>0.5828</td><td>0.1863</td><td>0.4316</td><td>0.55</td></tr>
<tr><td>Ensemble</td><td>8</td><td>8</td><td>0.5855</td><td>0.1864</td><td>0.4346</td><td>0.5502</td></tr>
<tr><td>Ensemble</td><td>9</td><td>9</td><td>0.586</td><td>0.186</td><td>0.4355</td><td>0.5492</td></tr>
<tr><td>Ensemble</td><td>10</td><td>10</td

# Variational inference with TF PROBA

In [32]:
import tensorflow_probability as tfp
from tensorflow_probability.python.layers import util as tfp_layers_util
from tensorflow_probability.python.distributions import kullback_leibler as kl_lib
from tensorflow_probability import distributions as tfd

kernel_posterior_scale_mean=-9.0
kernel_posterior_scale_stddev=0.1
kernel_posterior_scale_constraint=0.2

def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)
def approximate_kl(q, p, q_tensor):
    return tf.reduce_mean(q.log_prob(q_tensor) - p.log_prob(q_tensor))
def _untransformed_scale_constraint(t):
     # value used: https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/models/bayesian_vgg.py
    return tf.clip_by_value(t, -1000,tf.math.log(kernel_posterior_scale_constraint))

total_samples = len(train_images)
divergence_fn = lambda q, p, q_tensor : approximate_kl(q, p, q_tensor) / total_samples

def prior_trainable(kernel_size, bias_size=0, dtype=None):
  # Specify the prior over keras.layers.Dense kernel and bias.
  n = kernel_size + bias_size
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t, scale=1),
          reinterpreted_batch_ndims=1)),
  ])
# Specify the surrogate posterior over `keras.layers.Dense` `kernel` and `bias`.
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
  n = kernel_size + bias_size
  c = np.log(np.expm1(1.))
  return tf.keras.Sequential([
      tfp.layers.VariableLayer(2 * n, dtype=dtype),
      tfp.layers.DistributionLambda(lambda t: tfd.Independent(
          tfd.Normal(loc=t[..., :n],
                     scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
          reinterpreted_batch_ndims=1)),
  ])
kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn(
    untransformed_scale_initializer=tf.compat.v1.initializers.random_normal(
        mean=kernel_posterior_scale_mean,
        stddev=kernel_posterior_scale_stddev),
    untransformed_scale_constraint=_untransformed_scale_constraint)
  
def tfp_model(nb_output=10,two_layer=True):
    #model = models.Sequential()# https://www.tensorflow.org/tutorials/images/cnn
    inpx=layers.Input(shape=(32, 32, 3))
    x=layers.Conv2D(32, (3, 3), activation='relu')(inpx)
    x=layers.MaxPooling2D((2, 2))(x)
    x=layers.Conv2D(64, (3, 3), activation='relu')(x)
    x=layers.MaxPooling2D((2, 2))(x)
    x=layers.Conv2D(64, (3, 3), activation='relu')(x)
    x=layers.Flatten()(x)
    
    # 2 mode: variatonal mode and stand mode
    if two_layer:
      x=tfp.layers.DenseReparameterization(
          units = 64, activation = 'LeakyReLU',
          kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn(is_singular=False),
          kernel_prior_fn = tfp.layers.default_multivariate_normal_fn,
          bias_prior_fn = tfp.layers.default_multivariate_normal_fn,
          bias_posterior_fn = tfp.layers.default_mean_field_normal_fn(is_singular=False),
          kernel_divergence_fn = divergence_fn,
          bias_divergence_fn = divergence_fn
      )(x)
    else:
      x=layers.Dense(64, activation='relu')(x)
    
    x=tfp.layers.DenseReparameterization( #https://towardsdatascience.com/uncertainty-in-deep-learning-bayesian-cnn-tensorflow-probability-758d7482bef6
        units = tfp.layers.OneHotCategorical.params_size(nb_output), activation = None,
        kernel_posterior_fn = tfp.layers.default_mean_field_normal_fn(is_singular=False),
        kernel_prior_fn = tfp.layers.default_multivariate_normal_fn,
        bias_prior_fn = tfp.layers.default_multivariate_normal_fn,
        bias_posterior_fn = tfp.layers.default_mean_field_normal_fn(is_singular=False),
        kernel_divergence_fn = divergence_fn,
        bias_divergence_fn = divergence_fn
    )(x)
    model=Model(inpx,x)
    
    def myloss(y,y_):
      neg_log_likelihood = tf.nn.softmax_cross_entropy_with_logits(
      labels=y, logits=y_)
      kl=sum(model.losses)
      loss = neg_log_likelihood + 0.001*kl
      return loss
    model.compile(optimizer=tf.optimizers.Adam(), loss=myloss, metrics=['accuracy'])
    history = model.fit(train_images, train_labels, epochs=NB_EPOCHS, validation_data=(test_images, test_labels), verbose=VERBOSITY)
    return model

In [34]:
for two_varia_layer in [True, False]:
  # train
  model=tfp_model(two_layer=two_varia_layer)

  # predict
  global_preds=np.zeros(test_labels.shape)
  for i in range(64):
    logit=model.predict(test_images,verbose=VERBOSITY)
    global_preds+=softmax(logit)
    if (i+1)==4 or (i+1)==16 or (i+1)==64:
      score_info=evaluate(test_labels,global_preds.astype(np.float32)/(i+1.))
      print(f"<tr><td>{two_varia_layer}</td><td>1</td><td>{i+1}</td>{score_info}</tr>")

<tr><td>True</td><td>1</td><td>4</td><td>0.554</td><td>0.1949</td><td>0.3989</td><td>0.5749</td></tr>
<tr><td>True</td><td>1</td><td>16</td><td>0.5587</td><td>0.1942</td><td>0.404</td><td>0.5729</td></tr>
<tr><td>True</td><td>1</td><td>64</td><td>0.561</td><td>0.194</td><td>0.4064</td><td>0.5722</td></tr>
<tr><td>False</td><td>1</td><td>4</td><td>0.5268</td><td>0.2046</td><td>0.3786</td><td>0.605</td></tr>
<tr><td>False</td><td>1</td><td>16</td><td>0.5278</td><td>0.2037</td><td>0.3799</td><td>0.6021</td></tr>
<tr><td>False</td><td>1</td><td>64</td><td>0.53</td><td>0.2036</td><td>0.3822</td><td>0.6019</td></tr>


# TEMPERATURE SCALING<br>
URL:https://arxiv.org/pdf/1706.04599.pdf

In [ ]:
for training_rate in [0.1, 0.01, 0.001]:
  # get training/validation split data
  p=int(len(train_images)*(1-training_rate))
  model=keras_model_builder(train_images[:p], train_labels[:p])
  
  # get predictions
  valid_raw_y_pred=model.predict(train_images[p:],verbose=VERBOSITY)
  test_raw_y_pred=model.predict(test_images,verbose=VERBOSITY)

  del model

  # fit the temperature parameter
  id_criterion=1 # LLN criteria, like the linked paper on temperature scale
  valid_scores=[]
  info=[]
  for i in range(-10,30+1,1):
    t=1.2**float(i)
    valid_y_pred=softmax(softmax(valid_raw_y_pred)*t)
    test_y_pred=softmax(softmax(test_raw_y_pred)*t)
    
    valid_info_html, valid_info_array=evaluate(train_labels[p:], valid_y_pred, out_type="both")
    test_info_html, test_info_array=evaluate(test_labels, test_y_pred, out_type="both")

    if not np.isnan(float(valid_info_array[id_criterion])):
      valid_scores.append( valid_info_array[id_criterion] )
    info.append( f"<tr><td>{training_rate}</td><td>1</td><td>1</td>{test_info_html}</tr>" )

  # display the result
  best_temp_id=np.argmin(valid_scores)
  print(info[best_temp_id])